In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/cutnishant/diamonds.csv
/kaggle/input/testing-vidhya/test_QoiMO9B.csv
/kaggle/input/prayas/train.csv
/kaggle/input/vidhyalankar/train.csv
/kaggle/input/vidhyalankar/fulfilment_center_info.csv
/kaggle/input/vidhyalankar/meal_info.csv


In [28]:
import pandas as pd
x=str(input('What type of separation is there in the file such as (,) or (;)'))
train = pd.read_csv("/kaggle/input/cutnishant/diamonds.csv",sep=x)
# train.drop(columns='Technology List',inplace=True)

What type of separation is there in the file such as (,) or (;),


In [29]:
x=[]
for i in train.columns:
    if re.search('date', i, re.IGNORECASE):
        x.append(i)
columns=x

In [30]:
date=pd.DataFrame()
string=train.select_dtypes(include='object')
string.drop(columns=columns,inplace=True)
integer=train.select_dtypes(include=['int','float'])
date = pd.DataFrame(train, columns=columns)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
for i in date:
    date[i]= pd.to_datetime(date[i],yearfirst=True) 



In [32]:
date1=pd.DataFrame(date,columns=columns)
for i in date1:
    x=[]
    for j in date1[i]:
        x.append(int(j.strftime('%Y')))
    date[i+'_year']=x
for i in date1:
    x=[]
    for j in date1[i]:
        x.append(int(j.strftime('%m')))
    date[i+'_month']=x
for i in date1:
    x=[]
    for j in date1[i]:
        x.append(int(j.strftime('%d')))
    date[i+'_day']=x
for i in date1:
    x=[]
    for j in date1[i]:
        x.append(int(j.strftime('%H')))
    date[i+'_hour']=x
for i in date1:
    x=[]
    for j in date1[i]:
        x.append(int(j.strftime('%M')))
    date[i+'_minutes']=x
for i in date1:
    x=[]
    for j in date1[i]:
        x.append(int(j.strftime('%S')))
    date[i+'_second']=x
date.drop(columns=columns,inplace=True)
del date1
del x

In [33]:
# integer.isnull()
for i in integer.columns:
    integer[i].fillna(integer[i].mean(),inplace=True)


/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [34]:
string_1=string

In [35]:
x=[]
for i in string_1.columns:
    string_1[i] = string_1[i].str.replace(r'\D', '')
    try:
        string_1[i] = string_1[i].astype('int')
    except:
        x.append(i)
string_1.drop(columns=x,inplace=True)
string=pd.DataFrame(train,columns=x)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
for i in string.columns:
    string[i].fillna(string[i].mode().iloc[0],inplace=True)

In [37]:
for i in string.columns:
    x=[]
    arr=string[i].unique()
    n=string[i].nunique()
    for j in string[i]:
        for k in range(0,n):
            if(j==arr[k] ):
                x.append(k)
            else:
                continue
    string[i]=x
del x
del i
del train

In [38]:
train = integer.join(string, how='inner')
train = train.join(string_1, how='inner')
train = train.join(date, how='inner')

In [39]:
del integer
del string_1
del string
del date

In [40]:
import statsmodels.api as sm
def backward_elimination(data, target,significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant.astype(float)).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

In [41]:
x=str(input('Which is the column you want to predict?'))
x_train=train.drop(columns=x)
y_train=train[x]
columns=backward_elimination(x_train,y_train)
x_train = pd.DataFrame(x_train, columns=columns)
x_train.columns

Which is the column you want to predict?.


KeyError: "['.'] not found in axis"